# 🧩 Project Overview  

## **Title**  
**Discovering Hidden Barriers to Healthcare Access through Social Listening**

---

## **Goal**  
Use social media conversations (Reddit: *r/BabyBumps, r/pregnant, r/pregnantover35, r/diabetes, r/diabetes_t1, r/diabetes_t2, r/GestationalDiabetes, r/AskDocs, r/HealthInsurance*) to identify hidden barriers that prevent people from seeking care — such as cost, distance, stigma, or bureaucracy — and explore whether those barriers appear to causally influence intent to seek vs. avoid care.

---

## **Value in Addressing**  
Millions share candid health experiences online, yet data remains largely untapped for understanding real-world access challenges.  
By combining unsupervised learning (topic modeling, clustering, and network analysis) with causal inference (estimating whether certain discussions precede shifts in health-seeking intent), this project aims to connect online discourse to public health insights that are scalable, reproducible, and ethically grounded.

---

## **Data Sources**  
Reddit communities collected using the PRAW API:  
*r/BabyBumps, r/pregnant, r/pregnantover35, r/diabetes, r/diabetes_t1, r/diabetes_t2, r/GestationalDiabetes, r/AskDocs, r/HealthInsurance*

---

### **Selected Subreddit Groups**

| Subreddit Group           | Focus                             | Description of Group                                                                 |
| -------------------------- | --------------------------------- | ------------------------------------------------------------------------------------ |
| **r/BabyBumps**           | General pregnancy journey         | Community for expectant mothers sharing experiences, emotional narratives, and challenges related to pregnancy care and prenatal access. |
| **r/pregnant**            | Early-stage pregnancy experiences | Discussion forum covering early pregnancy, first-time parent concerns, high-risk pregnancies, and barriers to accessing prenatal healthcare. |
| **r/pregnantover35**      | Advanced maternal age pregnancy   | Support and discussion group for individuals 35+ navigating pregnancy, fertility, and higher-risk maternal care access challenges. |
| **r/GestationalDiabetes** | Pregnancy-specific diabetes       | Focused on individuals managing gestational diabetes, highlighting the intersection between maternal health and chronic disease management. |
| **r/diabetes**            | General diabetes management       | Broad community addressing chronic disease care, treatment costs, insurance barriers, and ongoing access to healthcare resources. |
| **r/diabetes_t1**         | Type 1 diabetes experiences       | Specialized forum emphasizing the experiences of those with Type 1 diabetes, including insulin affordability, medical technology, and self-management routines. |
| **r/diabetes_t2**         | Type 2 diabetes experiences       | Community focused on lifestyle management, medication access, affordability, and comorbidities associated with Type 2 diabetes. |
| **r/AskDocs**             | Direct health-seeking behavior    | Platform where users seek medical guidance from licensed healthcare professionals, providing insight into real-world intent to seek versus avoid care. |
| **r/HealthInsurance**     | Policy and access navigation      | Community focused on understanding and navigating insurance coverage, denials, affordability, and systemic healthcare access challenges. |

---

## **Supplementary Public Health Datasets (for optional validation)**  
- CDC Behavioral Risk Factor Surveillance System (BRFSS)  
- HRSA Health Resources Data  
- Kaiser Family Foundation (KFF) Health Access Indicators  

These datasets may be used to validate observed trends (e.g., cost or insurance-related barriers) against regional or national health access metrics.  

---

## **Analytic Approach**  
- Unsupervised Learning: Topic modeling, clustering, and barrier co-occurrence network mapping  
- Temporal Trend & Causal Inference: Granger analysis on intent shifts  
- Outcome Focus: Identifying causal relationships between language patterns and healthcare-seeking behavior

---

## **Key Questions to Answer**
1. Which barriers are most frequently cited as reasons for avoiding or delaying care?  
2. Do certain barriers cluster together (e.g., stigma + cost)?  
3. Which linguistic patterns signal implied causality in patient narratives, and how do these patterns align with observed temporal shifts in intent and behavior?
4. Are there hidden or unreported barriers (e.g., partner/family pressure, work schedule conflicts)?  
5. Do different communities (e.g., diabetes vs. pregnancy) experience distinct access challenges?

---


# Data Collection

In [ ]:
!pip install praw

In [ ]:
from google.colab import userdata
import praw

reddit = praw.Reddit(
    client_id=userdata.get("CLIENT_ID"),
    client_secret=userdata.get("CLIENT_SECRET"),
    user_agent=userdata.get("USER_AGENT"),
    check_for_async=False,
)

print("Read-only mode:", reddit.read_only)

Read-only mode: True


In [ ]:
import pandas as pd
from datetime import datetime

#Add Subreddit groups to pull
subreddits = ["BabyBumps", "pregnant", "diabetes", "GestationalDiabetes", "diabetes_t1", "diabetes_t2", "pregnantover35", "AskDocs", "HealthInsurance"]
all_posts = []

for sub in subreddits:
    print(f"Collecting from r/{sub}...")
    subreddit = reddit.subreddit(sub)
    for post in subreddit.top(time_filter="year", limit=7000):
        if post.selftext in ["[removed]", "[deleted]", None, ""]:
            continue
        all_posts.append({
            "subreddit": sub,
            "id": post.id,
            "title": post.title,
            "text": post.selftext,
            "created_utc": post.created_utc,
            "created": datetime.utcfromtimestamp(post.created_utc)
        })

/tmp/ipython-input-553858498.py:20: DeprecationWarning:

datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).



In [ ]:
#Create dataframe and print
df = pd.DataFrame(all_posts)
print(df["subreddit"].value_counts())
df.to_parquet("reddit_raw.parquet", index=False)

# Data Cleaning

In [ ]:
import re

def clean_text(text):
    text = str(text) if text is not None else ""
    text = re.sub(r"http\S+", " ", text)        # remove URLs
    text = re.sub(r"[\r\n]+", " ", text)        # remove newlines
    text = re.sub(r"\s{2,}", " ", text)         # collapse spaces
    return text.strip().lower()

# Combine title + body
df["text_full"] = (df["title"].fillna("") + " " + df["text"].fillna("")).str.strip()

# Create cleaned text
df["text_clean"] = df["text_full"].apply(clean_text)

# Optional: drop very short posts
df = df[df["text_clean"].str.len() > 50].copy()

print(df.columns)
df[["subreddit", "text_clean"]].head()

# Barrier & Intent Tagging

In [ ]:
# Barrier keyword dictionaries
barriers = {
    "cost": [
        "can't pay", "cannot afford", "couldn't afford",
        "insurance denied", "insurance won't cover",
        "copay", "co-pay",
        "bills", "medical bills",
        "coverage", "lost my coverage",
        "expensive", "too expensive", "too costly",
        "price", "costs too much",
        "out of pocket", "pay out of pocket",
        "deductible", "high deductible",
        "can't afford strips", "can't afford insulin", "can't afford cgm",
        "budget", "financial", "money",
        "unaffordable", "no insurance", "uninsured",
        "out of network",
        "medical debt", "financially struggling",
        "prescription too expensive"
    ],

    "geo": [
        "too far to travel",
        "no transport", "no transportation", "don't have transportation",
        "transportation issues",
        "can't drive", "no ride", "no car",
        "miles away", "far away", "hours away",
        "long drive", "drive hours",
        "can't get there", "can't get to the clinic",
        "clinic too far", "clinic is too far",
        "nearest doctor is far", "nearest is", "closest clinic",
        "rural", "rural area",
        "distance to hospital",
        "travel time", "travel costs",
        "don't live near", "not available in my area"
    ],

    "bureau": [
        "paperwork", "paper work",
        "waitlist", "waiting list",
        "appointment backlog",
        "denied", "denied referral",
        "referral", "can't get a referral",
        "authorization", "prior authorization",
        "red tape",
        "can't get through", "on hold",
        "weeks wait", "months wait", "appointment is months out",
        "couldn't get in", "no availability", "booked out",
        "can't get an appointment", "unable to schedule",
        "impossible to get",
        "gave up calling",
        "got lost in the system",
        "confusing process",
        "don't know where to go",
        "can't find provider",
        "system won't help"
    ],

    "stigma": [
        "stigma", "fear of stigma",
        "embarrassed", "embarrassing",
        "ashamed", "ashamed to ask",
        "shame", "guilty", "blamed myself", "feel like a failure",
        "feel judged", "feeling judged",
        "afraid to ask",
        "don't want people to know", "hiding it", "keeping it secret",
        "scared to tell",
        "family doesn't understand", "family does not understand",
        "partner thinks", "people will think",
        "privacy concerns",
        "afraid to talk about it",
        "worried about discrimination",
        "scared of what doctor will say"
    ],

    "language_info": [
        "language barrier",
        "don't understand instructions",
        "don't understand what they said",
        "too much medical jargon",
        "can't read paperwork",
        "can't read the forms",
        "not explained clearly",
        "hard to follow directions",
        "confusing information",
        "translation issue", "interpreter needed",
        "no materials in my language"
    ],

    "time_work": [
        "can't get time off",
        "can't take time off",
        "work won't let me",
        "no sick leave",
        "no sick days",
        "clinic not open when i can go",
        "clinic not open when I can go",
        "too busy for appointment",
        "too busy to see a doctor",
        "have to work during hours",
        "conflicts with work",
        "can't schedule around job",
        "work schedule conflict",
        "shift work makes it hard",
        "no time for appointment"
    ],

    "trust_mistrust": [
        "don't trust doctor", "do not trust doctor",
        "don't trust hospital", "do not trust hospital",
        "don't trust healthcare", "do not trust healthcare",
        "don't trust medical advice",
        "lost faith in system",
        "healthcare failed me",
        "bad experience before",
        "had a bad experience",
        "wasn't listened to", "was not listened to",
        "doctor ignored me",
        "didn't feel heard", "did not feel heard",
        "they didn't believe me",
        "they don't believe me"
    ]
}

# Care trajectory / intent keywords

barrier_overcome = [
    "finally got", "was able to", "managed to get",
    "insurance approved", "found a way", "worth it in the end",
    "glad I went", "should have gone sooner", "wish I hadn't waited"
]

intent_seek = [
    "booked an appointment", "booked appointment",
    "made an appointment", "have an appointment",
    "seeing my doctor", "seeing the doctor",
    "scheduled a visit", "going next week",
    "going to the doctor", "finally scheduled",
    "got an appointment", "set up appointment",
    "went to see", "visited the doctor",
    "started seeing",
    "got referred", "making an appointment",
    "decided to go"
]

intent_considering = [
    "worth it to see",
    "considering", "considering going",
    "should I", "should i",
    "is it necessary",
    "thinking about", "thinking about going",
    "wondering if I should", "wondering if i should",
    "debating whether"
]

intent_delayed = [
    "bit the bullet",
    "eventually",
    "putting off",
    "finally went",
    "took me weeks",
    "waited too long",
    "delayed"
]

intent_ambivalent = [
    "not sure if",
    "don't know if I should", "don't know if i should",
    "on the fence",
    "hesitant to",
    "reluctant to"
]

intent_avoid = [
    "not going to the doctor", "not going",
    "avoiding the doctor", "avoiding doctors",
    "not worth it",
    "won't go", "will not go",
    "cant go", "can't go",
    "skip the appointment", "skipping",
    "skipped appointment",
    "cancelled appointment", "canceled appointment",
    "rather not",
    "didn't go", "did not go",
    "not bothering",
    "gave up trying",
    "too much hassle",
    "managing on my own",
    "doing it myself"
]

In [ ]:
# Tagging functions

def tag_barriers(txt):
    """
    Return a list of barrier categories present in the text.
    Uses simple substring matching; each barrier type is only tagged once.
    """
    txt = txt.lower()
    found = []
    for k, words in barriers.items():
        # Sort longest phrases first to favor more specific matches
        for w in sorted(words, key=len, reverse=True):
            if w.lower() in txt:
                found.append(k)
                break  # Only flag each barrier type once
    # Remove duplicates just in case
    return list(set(found))


def tag_intent(txt):
    """
    Return a single intent label based on priority:
    avoid > delayed > ambivalent > considering > seek
    """
    txt = txt.lower()

    for w in sorted(intent_avoid, key=len, reverse=True):
        if w.lower() in txt:
            return "avoid"

    for w in sorted(intent_delayed, key=len, reverse=True):
        if w.lower() in txt:
            return "delayed"

    for w in sorted(intent_ambivalent, key=len, reverse=True):
        if w.lower() in txt:
            return "ambivalent"

    for w in sorted(intent_considering, key=len, reverse=True):
        if w.lower() in txt:
            return "considering"

    for w in sorted(intent_seek, key=len, reverse=True):
        if w.lower() in txt:
            return "seek"

    return None


def tag_barrier_overcome(txt):
    """
    Return True if any 'barrier_overcome' phrase is present.
    """
    txt = txt.lower()
    for w in sorted(barrier_overcome, key=len, reverse=True):
        if w.lower() in txt:
            return True
    return False


# Apply tagging to the dataframe

print("Tagging barriers...")
df["barriers"] = df["text_clean"].apply(tag_barriers)

print("Tagging intent...")
df["intent"] = df["text_clean"].apply(tag_intent)

print("Tagging barrier_overcome...")
df["barrier_overcome"] = df["text_clean"].apply(tag_barrier_overcome)

# --- Create barrier flags (one per category + overall) ---

df["has_cost"]           = df["barriers"].apply(lambda x: "cost" in x)
df["has_geo"]            = df["barriers"].apply(lambda x: "geo" in x)
df["has_bureau"]         = df["barriers"].apply(lambda x: "bureau" in x)
df["has_stigma"]         = df["barriers"].apply(lambda x: "stigma" in x)
df["has_language_info"]  = df["barriers"].apply(lambda x: "language_info" in x)
df["has_time_work"]      = df["barriers"].apply(lambda x: "time_work" in x)
df["has_trust_mistrust"] = df["barriers"].apply(lambda x: "trust_mistrust" in x)

df["has_any_barrier"]    = df["barriers"].apply(lambda x: len(x) > 0)
df["barrier_count"]      = df["barriers"].apply(len)

# Barrier & Intent Analysis

In [ ]:
import pandas as pd

# Analyze and create comprehensive reporting
print("\n" + "=" * 50)
print("BARRIER OVERLAP ANALYSIS")
print("=" * 50)

# Posts with ONLY a single barrier type
only_cost = df["barriers"].apply(lambda x: (len(x) == 1) and ("cost" in x))
only_bureau = df["barriers"].apply(lambda x: (len(x) == 1) and ("bureau" in x))

print(f"\nPosts with ONLY cost barriers: {only_cost.sum()}")
print(f"Posts with ONLY bureau barriers: {only_bureau.sum()}")

# Posts with BOTH cost AND bureau (regardless of other barriers)
both_cost_bureau = df["barriers"].apply(lambda x: ("cost" in x) and ("bureau" in x))
print(f"Posts with BOTH cost AND bureau barriers: {both_cost_bureau.sum()}")

print("\n" + "=" * 50)
print("OVERALL STATISTICS")
print("=" * 50)

print(f"\nTotal posts: {len(df)}")
print(f"Posts with at least one barrier: {df['has_any_barrier'].sum()}")
print(f"Posts with no barriers: {(~df['has_any_barrier']).sum()}")
print(f"Posts with multiple barriers: {df['barriers'].apply(lambda x: len(x) > 1).sum()}")
print(f"Average barriers per post: {df['barriers'].apply(len).mean():.2f}")

print("\n" + "=" * 50)
print("BARRIER TYPE COUNTS")
print("=" * 50)
print(df["barriers"].explode().value_counts())

print("\n" + "=" * 50)
print("INTENT COUNTS")
print("=" * 50)
print(df["intent"].value_counts(dropna=False))

print("\n" + "=" * 50)
print("BARRIER + INTENT CROSSTAB")
print("=" * 50)
print(pd.crosstab(df['has_any_barrier'], df['intent'], dropna=False, margins=True))

print("\n" + "=" * 50)
print("BARRIER COMBINATIONS (Top 10)")
print("=" * 50)

# Use all has_* barrier flags except has_any_barrier
barrier_flag_cols = [
    col for col in df.columns
    if col.startswith("has_") and col not in ["has_any_barrier"]
]

barrier_combos = df.groupby(barrier_flag_cols).size()
print(barrier_combos.sort_values(ascending=False).head(10))

print("\n" + "=" * 50)
print("CAUSAL INDICATORS")
print("=" * 50)

if df['has_any_barrier'].sum() > 0:
    avoid_with_barrier = df[df['has_any_barrier'] & (df['intent'] == 'avoid')].shape[0]
    total_with_barrier = df['has_any_barrier'].sum()
    print(f"Avoid rate WITH barriers: {avoid_with_barrier / total_with_barrier * 100:.1f}%")

if (~df['has_any_barrier']).sum() > 0:
    avoid_without_barrier = df[~df['has_any_barrier'] & (df['intent'] == 'avoid')].shape[0]
    total_without_barrier = (~df['has_any_barrier']).sum()
    print(f"Avoid rate WITHOUT barriers: {avoid_without_barrier / total_without_barrier * 100:.1f}%")

# Save tagged data
df.to_parquet("reddit_tagged.parquet", index=False)
print("\n✅ Data saved to reddit_tagged.parquet")

In [ ]:
# Run analysis to analyze posts by barrier type

barrier_types = [
    "cost",
    "geo",
    "bureau",
    "stigma",
    "language_info",
    "time_work",
    "trust_mistrust"
]

for barrier_type in barrier_types:
    flag_col = f"has_{barrier_type}"
    if flag_col not in df.columns:
        # Skip gracefully if a flag wasn't created for some reason
        continue

    subset = df[df[flag_col]]

    print(f"\n{barrier_type.upper()} barrier breakdown:")
    print(f"  Total posts: {len(subset)}")

    with_intent = subset["intent"].notna().sum()
    print(f"  With intent: {with_intent}")

    if with_intent > 0:
        print(subset["intent"].value_counts(dropna=False))

    if len(subset) > 0:
        avoid_rate = (subset["intent"] == "avoid").sum() / len(subset) * 100
        print(f"  Avoid rate: {avoid_rate:.1f}%")

# Sentiment Analysis

In [ ]:
!pip install nltk

In [ ]:
#Run sentiment analysis using Vader
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

nltk.download('vader_lexicon', quiet=True)
sid = SentimentIntensityAnalyzer()

# Create Sentiment column
df["sentiment_score"] = df["text"].fillna("").apply(
    lambda x: sid.polarity_scores(str(x))["compound"]
)

In [ ]:
# Sentiment distribution
print("\n" + "=" * 50)
print("SENTIMENT DISTRIBUTION")
print("=" * 50)
print(df["sentiment_score"].describe())

# Sentiment by label
def label_sentiment(score):
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"

df["sentiment_label"] = df["sentiment_score"].apply(label_sentiment)
print(df["sentiment_label"].value_counts())

# Average sentiment by barrier type
print("\n" + "=" * 50)
print("AVERAGE SENTIMENT BY BARRIER")
print("=" * 50)
for barrier in ["cost", "geo", "bureau", "stigma",
                "language_info", "time_work", "trust_mistrust"]:
    mask = df[f"has_{barrier}"]
    if mask.any():
        mean_score = df.loc[mask, "sentiment_score"].mean()
        print(f"{barrier:15}: {mean_score:6.3f}  (n={mask.sum()})")

# Average sentiment by intent
print("\n" + "=" * 50)
print("AVERAGE SENTIMENT BY INTENT")
print("=" * 50)
print(df.groupby("intent")["sentiment_score"].mean().sort_values())

# Unsupervised Models (LDA & BERTopic)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Vectorize text
texts = df["text_clean"].fillna("")

vectorizer = CountVectorizer(
    max_df=0.95,        # ignore very common words
    min_df=10,          # ignore very rare words
    stop_words="english"
)
dtm = vectorizer.fit_transform(texts)

# Fit LDA
n_topics = 10
lda = LatentDirichletAllocation(
    n_components=n_topics,
    random_state=42,
    learning_method="batch"
)
lda_doc_topic = lda.fit_transform(dtm)

# Assign dominant LDA topic per post
df["lda_topic"] = lda_doc_topic.argmax(axis=1)

In [ ]:
import numpy as np

#Top LDA topics
feature_names = vectorizer.get_feature_names_out()

def print_lda_topics(model, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        top_indices = topic.argsort()[::-1][:n_top_words]
        top_words = [feature_names[i] for i in top_indices]
        print(f"\nTopic {topic_idx}:")
        print(", ".join(top_words))

print_lda_topics(lda, feature_names, n_top_words=12)

In [ ]:
!pip install bertopic[visualization] umap-learn hdbscan

In [ ]:
from bertopic import BERTopic
print("BERTopic imported successfully ✅")

In [ ]:
# Initialize and fit the model
topic_model = BERTopic(language="english", calculate_probabilities=True)

# Fit to your cleaned text column (replace with your actual text column)
topics, probs = topic_model.fit_transform(df["text_clean"].dropna())

# Add results back to dataframe for analysis
df["topic"] = topics
df["topic_probability"] = probs.max(axis=1)

In [ ]:
#BERTopic analysis
print("\n" + "="*50)
print("BERTopic Summary")
print("="*50)

# Total topics (excluding -1 "outliers")
num_topics = len(set(df["topic"])) - (1 if -1 in df["topic"].unique() else 0)
print(f"Total topics (excluding outliers): {num_topics}")
print(f"Outlier posts: {(df['topic'] == -1).sum()}")

# Top 10 most frequent topics
print("\nTop 10 topics by frequency:")
print(df["topic"].value_counts().head(10))


In [ ]:
#Parameter tune for better output
topic_model = BERTopic(
    language="english",
    min_topic_size=20,      # smaller = more granular clusters
    nr_topics=None,         # keep dynamic, or set an upper limit
    n_gram_range=(1, 2),    # include bigrams
    calculate_probabilities=True
)
topics, probs = topic_model.fit_transform(df["text_clean"])

# Integrated Comparison

In [ ]:
# Get top keywords for each topic
topic_info = topic_model.get_topic_info()
print(topic_info.head(10))

# Display keywords for a specific topic (e.g., topic 0)
topic_id = 0
print(f"\nTop words for topic {topic_id}:")
print(topic_model.get_topic(topic_id))

# Show a few representative posts from that topic
sample_posts = df[df["topic"] == topic_id].sample(5)
print(sample_posts[["subreddit", "intent", "barriers", "sentiment_label", "text_clean"]])

In [ ]:
#Compare LDA with BERTopic + barriers/intent
import pandas as pd

print("\nLDA topic counts:")
print(df["lda_topic"].value_counts().head())

print("\nLDA topic × BERTopic topic (small sample):")
print(pd.crosstab(df["lda_topic"], df["topic"]).head())

print("\nAverage sentiment per LDA topic:")
print(df.groupby("lda_topic")["sentiment_score"].mean().sort_values())

print("\nAverage cost barrier presence per LDA topic:")
print(df.groupby("lda_topic")["has_cost"].mean().sort_values(ascending=False).head())

print("\nIntent by LDA topic:")
print(pd.crosstab(df["lda_topic"], df["intent"], normalize="index"))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Topic word embeddings coherence
topics_dict = topic_model.get_topics()
coherence_scores = {}
for t, words in topics_dict.items():
    if t == -1:  # Skip outliers
        continue
    embeddings = [topic_model.embedding_model.embed(word) for word, _ in words[:10]]
    sims = cosine_similarity(embeddings)
    coherence_scores[t] = np.mean(sims[np.triu_indices_from(sims, 1)])

print("\nApproximate topic coherence (top 10):")
print(sorted(coherence_scores.items(), key=lambda x: x[1], reverse=True)[:10])

In [ ]:
#Barrier & Intent Crossover by Topic
print("\n" + "="*50)
print("TOPIC vs BARRIER / INTENT RELATIONSHIP")
print("="*50)

# Barrier overlap by topic
for barrier in ["cost", "geo", "bureau", "stigma",
                "language_info", "time_work", "trust_mistrust"]:
    col = f"has_{barrier}"
    if col in df.columns:
        mean_val = df.groupby("topic")[col].mean().rename(f"{barrier}_rate")
        print(f"\nAverage {barrier} presence by topic:")
        print(mean_val.sort_values(ascending=False).head(10))

# Intent breakdown by topic
intent_ct = pd.crosstab(df["topic"], df["intent"], normalize="index")
print("\nIntent distribution by topic:")
print(intent_ct.head(10))

In [ ]:
#Sentiment Relationship by Topic
topic_sentiment = (
    df.groupby("topic")["sentiment_score"]
      .agg(["mean", "count"])
      .sort_values("mean")
)
print(topic_sentiment.head(10))

# Visualizations & Interpretations

In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Create figures directory
os.makedirs("figures", exist_ok=True)

In [ ]:
# From stats printout:
barriers = ["cost", "stigma", "bureau", "geo"]
counts   = [240, 89, 88, 32]

plt.figure(figsize=(10,5))
bars = plt.bar(barriers, counts, color=['#cc0000','#880000','#550000','#333333'])

# formatting
plt.title("Barrier Type Counts")
plt.ylabel("Number of Posts")
plt.xlabel("Barrier Type")

# Add labels above bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 3,
             '%d' % int(height), ha='center')

# Light horizontal grid
plt.grid(axis='y', alpha=0.3)

plt.show()

In [ ]:
# Topic and Barrier Trends over Time
df['month'] = df['created'].dt.to_period('M').astype(str)
barrier_trends = (
    df.groupby(['subreddit', 'month'])
      .agg(barrier_rate=('has_any_barrier', 'mean'),
           cost_rate=('has_cost', 'mean'),
           stigma_rate=('has_stigma', 'mean'),
           n_posts=('id', 'count'))
      .reset_index()
)

plt.figure(figsize=(12,6))
sns.lineplot(data=barrier_trends, x='month', y='barrier_rate', hue='subreddit')
plt.title("Barrier Mentions Over Time by Community")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("figures/barrier_trends_by_community.png", dpi=300)
plt.show()

In [ ]:
# Sentiment vs Intent
sent_intent = (
    df.groupby('intent')['sentiment_score']
      .mean()
      .reset_index()
      .sort_values('sentiment_score')
)

plt.figure(figsize=(8,5))
sns.barplot(data=sent_intent, x='intent', y='sentiment_score', palette='coolwarm')
plt.title("Average Sentiment by Health-Seeking Intent")
plt.ylabel("Mean Sentiment (VADER Compound)")
plt.xlabel("Intent Type")
plt.tight_layout()
plt.savefig("figures/sentiment_by_intent.png", dpi=300)
plt.show()

In [ ]:
# Heatmap of sentiment x barrier type
heatmap_data = {}
for barrier in ["cost", "geo", "bureau", "stigma", "language_info", "time_work", "trust_mistrust"]:
    mask = df[f"has_{barrier}"]
    if mask.any():
        heatmap_data[barrier] = (
            df.loc[mask]
              .groupby('intent')['sentiment_score']
              .mean()
              .reindex(["seek","considering","ambivalent","delayed","avoid"])
        )

plt.figure(figsize=(8,6))
sns.heatmap(pd.DataFrame(heatmap_data).T, annot=True, cmap="RdYlGn", center=0)
plt.title("Sentiment by Intent and Barrier Type")
plt.tight_layout()
plt.savefig("figures/sentiment_barrier_heatmap.png", dpi=300)
plt.show()

In [ ]:
# Causal Findings
panel = (
    df.groupby(['subreddit', 'month'])
      .agg(pct_avoid=('intent', lambda x: (x=='avoid').mean()),
           pct_seek=('intent', lambda x: (x=='seek').mean()),
           avg_sentiment=('sentiment_score','mean'),
           barrier_rate=('has_any_barrier','mean'))
      .reset_index()
)

plt.figure(figsize=(10,6))
sns.scatterplot(data=panel, x='barrier_rate', y='pct_avoid', hue='subreddit')
plt.title("Barrier Rate vs Avoidance Intent by Subreddit")
plt.xlabel("Barrier Rate (Proportion of Posts)")
plt.ylabel("Avoidance Intent Rate")
plt.tight_layout()
plt.savefig("figures/barrier_vs_avoidance_scatter.png", dpi=300)
plt.show()

In [ ]:
# Narrative Interpretation
print("\n🩺 INTERPRETATION SUMMARY")
print("="*70)
print("1️⃣ Cost and bureaucratic barriers show the strongest co-occurrence with 'avoid' intent.")
print("2️⃣ Pregnancy-related subreddits (BabyBumps, pregnant, GestationalDiabetes) show higher emotional tone and more stigma mentions.")
print("3️⃣ Chronic condition subreddits (diabetes_t1/t2) show steadier sentiment but higher cost discussions.")
print("4️⃣ Negative sentiment correlates moderately with avoidance intent across all communities.")
print("5️⃣ Overall, cost + stigma barriers appear to have the largest predictive effect on care-seeking delay or avoidance.")

In [ ]:
# ----- LDA Topic Visualization (Bar Chart of Top Words per Topic) -----

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

n_top_words = 8   # choose how many per topic
num_topics = lda.n_components

fig, axes = plt.subplots(
    nrows=num_topics // 2 + num_topics % 2,
    ncols=2,
    figsize=(14, num_topics * 2.2),
    constrained_layout=True
)

axes = axes.flatten()

for idx, topic in enumerate(lda.components_[:num_topics]):
    top_indices = topic.argsort()[::-1][:n_top_words]
    words = [vectorizer.get_feature_names_out()[i] for i in top_indices]
    weights = topic[top_indices]

    axes[idx].barh(words[::-1], weights[::-1], color="#7b9acc")
    axes[idx].set_title(f"Topic {idx}")
    axes[idx].invert_yaxis()

# Turn off unused subplots
for ax in axes[num_topics:]:
    ax.set_visible(False)

plt.suptitle("LDA Topics – Top Keywords", fontsize=18)
plt.show()

In [ ]:
# Barchart BERTopics
fig_sizes = topic_model.visualize_barchart(top_n_topics=20)
fig_sizes.show()
fig_sizes.write_html("figures/bertopic_topic_sizes.html")

In [ ]:
# --- LDA topic counts ---
lda_counts = df["lda_topic"].value_counts().sort_index()
lda_df = lda_counts.rename_axis("topic").reset_index(name="count")
lda_df["model"] = "LDA"

# --- BERTopic topic counts (exclude -1 = outliers) ---
bt_counts = df[df["topic"] != -1]["topic"].value_counts().sort_index()
bt_df = bt_counts.rename_axis("topic").reset_index(name="count")
bt_df["model"] = "BERTopic"

# Combine
topic_sizes = pd.concat([lda_df, bt_df], ignore_index=True)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

# LDA
axes[0].bar(lda_df["topic"].astype(str), lda_df["count"])
axes[0].set_title("LDA Topic Sizes")
axes[0].set_xlabel("LDA Topic")
axes[0].set_ylabel("Number of Posts")

# BERTopic
axes[1].bar(bt_df["topic"].astype(str), bt_df["count"])
axes[1].set_title("BERTopic Topic Sizes")
axes[1].set_xlabel("BERTopic Topic")

plt.tight_layout()
plt.show()

# Causal Inference

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

# Set datetime
df['created'] = pd.to_datetime(df['created'])

# Define subreddit groups
groups = {
    "pregnancy": [
        "BabyBumps", "pregnant", "pregnantover35", "GestationalDiabetes"
    ],
    "diabetes": [
        "diabetes", "diabetes_t1", "diabetes_t2"
    ],
    "access_insurance": [
        "AskDocs", "HealthInsurance"
    ]
}

# Helper: create monthly panel for a group
def make_group_panel(sub_list, label):
    tmp = df[df['subreddit'].isin(sub_list)].copy()
    if tmp.empty:
        print(f"[WARN] No posts found for group: {label}")
        return None

    tmp['month'] = tmp['created'].dt.to_period('M').dt.to_timestamp()  # month start

    panel = (
        tmp.groupby('month')
           .agg(
               barrier_rate=('has_any_barrier', 'mean'),
               cost_rate=('has_cost', 'mean'),
               avoid_rate=('intent', lambda x: (x == 'avoid').mean()),
               seek_rate=('intent', lambda x: (x == 'seek').mean()),
               avg_sentiment=('sentiment_score', 'mean'),
               n_posts=('id', 'count')
           )
           .sort_index()
    )
    panel['group'] = label
    return panel

In [ ]:
# Build panels for all three groups
panels = {}
for label, subs in groups.items():
    panel = make_group_panel(subs, label)
    if panel is not None:
        # Regularize to monthly frequency and interpolate small gaps
        panel = panel.asfreq('MS').interpolate()
        panels[label] = panel
        print(f"{label}: {len(panel)} monthly observations")

# Helper: Granger wrapper
def run_granger(ts_df, x_col, y_col, maxlag=3, title=""):
    """
    Test whether x_col Granger-causes y_col in ts_df.
    ts_df must be indexed by time and contain columns [x_col, y_col].
    """
    print("\n" + "="*80)
    print(f"GRANGER CAUSALITY TEST: {x_col} → {y_col} {title}")
    print("="*80)

    data = ts_df[[y_col, x_col]].dropna()  # order: [y, x]
    if len(data) < (maxlag*2 + 1):
        print(f"Not enough data points for maxlag={maxlag}. Only {len(data)} rows.")
        return

    results = grangercausalitytests(data, maxlag=maxlag, verbose=False)
    for lag in range(1, maxlag+1):
        p_val = results[lag][0]['ssr_ftest'][1]
        print(f"Lag {lag}: p-value = {p_val:.4f}")
    print("Note: p-values < 0.05 suggest x helps predict future y (Granger-causal).")

In [ ]:
# Run Granger for each group and predictor
maxlag = 3
for label, panel in panels.items():
    # Any barrier -> avoidance
    run_granger(
        panel,
        x_col="barrier_rate",
        y_col="avoid_rate",
        maxlag=maxlag,
        title=f"({label} communities)"
    )

    # Cost barrier -> avoidance
    run_granger(
        panel,
        x_col="cost_rate",
        y_col="avoid_rate",
        maxlag=maxlag,
        title=f"({label} communities, cost barriers)"
    )

# Causal Visualizations

In [ ]:
# Causal Visualizations: Barriers vs Avoidance Over Time

sns.set(style="whitegrid")
os.makedirs("figures", exist_ok=True)

# Combine all group panels into a single DataFrame
combined_panel = []

for label, panel in panels.items():
    tmp = panel.copy()
    tmp = tmp.reset_index().rename(columns={"month": "date"})
    tmp["group"] = label
    combined_panel.append(tmp)

combined_panel = pd.concat(combined_panel, ignore_index=True)
combined_panel.head()

In [ ]:
# Line plot: barrier_rate and avoid_rate by group over time

plt.figure(figsize=(12, 6))
sns.lineplot(
    data=combined_panel,
    x="date", y="barrier_rate",
    hue="group", marker="o"
)
plt.title("Barrier Rate Over Time by Community Group")
plt.xlabel("Month")
plt.ylabel("Barrier Rate (proportion of posts)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("figures/barrier_rate_over_time_by_group.png", dpi=300)
plt.show()

plt.figure(figsize=(12, 6))
sns.lineplot(
    data=combined_panel,
    x="date", y="avoid_rate",
    hue="group", marker="o"
)
plt.title("Avoidance Intent Over Time by Community Group")
plt.xlabel("Month")
plt.ylabel("Avoidance Rate (proportion of posts)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("figures/avoid_rate_over_time_by_group.png", dpi=300)
plt.show()

In [ ]:
# FacetGrid: barrier_rate and avoid_rate over time, per group
# Melt to long format to plot both series in one facet per group
long_panel = combined_panel.melt(
    id_vars=["date", "group"],
    value_vars=["barrier_rate", "avoid_rate"],
    var_name="metric",
    value_name="value"
)

g = sns.relplot(
    data=long_panel,
    x="date", y="value",
    hue="metric",
    col="group",
    kind="line",
    marker="o",
    facet_kws={"sharey": False, "sharex": True},
    height=4, aspect=1.2
)
g.set_titles("{col_name} communities")
g.set_xlabels("Month")
g.set_ylabels("Rate")
for ax in g.axes.flatten():
    for label in ax.get_xticklabels():
        label.set_rotation(45)
        label.set_ha("right")

plt.tight_layout()
plt.savefig("figures/barrier_vs_avoid_facet_by_group.png", dpi=300)
plt.show()

# Sample Posts

In [ ]:
# Function to anonymize & shorten text
def anonymize_sample(text, max_len=250):
    text = str(text)
    # Remove URLs
    text = re.sub(r"http\S+", "", text)
    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    # Shorten long posts
    if len(text) > max_len:
        text = text[:max_len] + "..."
    return text

# Select posts that actually HAVE content + a barrier or intent
sample_df = df[
    (df["text_clean"].str.len() > 80) &
    (df["barriers"].apply(len) > 0) |
    (df["intent"].notna())
]

# Sample N posts
N = 5
samples = sample_df.sample(N, random_state=42)

# Create anonymized output
samples_display = samples[[
    "subreddit", "barriers", "intent", "sentiment_label", "text_clean"
]].copy()

samples_display["excerpt"] = samples_display["text_clean"].apply(anonymize_sample)

# Show final output for screenshots
samples_display = samples_display[[
    "subreddit", "barriers", "intent", "sentiment_label", "excerpt"
]]

samples_display